<a href="https://colab.research.google.com/github/danrosher/colab/blob/main/Vehicle_Orientation_using_optimal_params.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://gitlab.com/rosherd/vehicle_orient_deeplearning.git

In [ ]:
%tensorflow_version 2.x



In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import matplotlib.pylab as plt
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
import numpy as np
import PIL.Image as Image
from tensorflow.keras import regularizers
from keras.callbacks import TensorBoard
from keras import optimizers
from tensorboard.plugins.hparams import api as hp


In [ ]:

IMAGE_SIZE=224
validation_split=0.1

IMAGE_SHAPE = (IMAGE_SIZE, IMAGE_SIZE)

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255,
    validation_split=validation_split)

train_data_dir = 'vehicle_orient_deeplearning/vehicle_images' 

train_generator = train_datagen.flow_from_directory(
    train_data_dir+"/all",
    target_size=IMAGE_SHAPE,
    shuffle=True,
    class_mode='categorical',
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    train_data_dir+"/all",
    target_size=IMAGE_SHAPE,
    shuffle=True,
    class_mode='categorical',
    subset='validation') # set as validation data

In [ ]:
steps_per_epoch = np.ceil(train_generator.samples/train_generator.batch_size)
validation_steps_per_epoch = np.ceil(validation_generator.samples/validation_generator.batch_size)

input_tensor = tf.keras.layers.Input(shape=(224, 224, 3),name='input_image')
image_model = tf.keras.applications.InceptionV3(include_top=False,  weights='imagenet')

image_model.trainable = True
fine_tune_at = 120
# Freeze all the layers before the `fine_tune_at` layer
for layer in image_model.layers[:fine_tune_at]:
  layer.trainable =  False

model = tf.keras.Sequential([ 
  input_tensor,
  image_model,
  layers.GlobalAveragePooling2D(),
  layers.Dense(hparams['HP_NUM_UNITS'], activation=tf.nn.relu,kernel_regularizer=tf.keras.regularizers.l2(0.0001)),
  layers.Dropout(hparams['HP_DROPOUT']),
  layers.Dense(train_generator.num_classes, name='out', activation='softmax',kernel_regularizer=tf.keras.regularizers.l2(hparams['HP_L2']))
])

hparams = {
    'HP_L2' : 0.0001,
    'HP_OPTIMIZER' : tf.keras.optimizers.SGD(lr=0.1),
    'HP_NUM_UNITS' : 64,
    'HP_DROPOUT' : 0.06
}

model.compile(
    optimizer=hparams['HP_OPTIMIZER'],
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy'])

model.summary()

history = model.fit_generator(
  train_generator,
  epochs=1,
  steps_per_epoch=steps_per_epoch,
  validation_data = validation_generator, 
  validation_steps = validation_steps_per_epoch
)


In [ ]:
history = model.fit_generator(
  train_generator,
  epochs=10,
  steps_per_epoch=steps_per_epoch,
  validation_data = validation_generator, 
  validation_steps = validation_steps_per_epoch
)

In [ ]:

from matplotlib import pyplot

# plot learning curves
pyplot.title('Learning Curves')
pyplot.xlabel('Epoch')
pyplot.ylabel('Cross Entropy')
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='val')
pyplot.legend()
pyplot.show()


In [ ]:
!mkdir saved_model
export_path = "saved_model"
model.save(export_path, save_format='tf')
!tar cvzf /content/model.tgz /content/saved_model

In [ ]:
!saved_model_cli show --dir {export_path} --all